In [3]:
# Milestone 1: Data Collection & Preparation
# Azure Demand Forecasting Project

import pandas as pd
import numpy as np

print("Starting Milestone 1: Data Collection & Preparation\n")

# Load the dataset
df = pd.read_excel("Azure_Enterprise_Realistic_Custom_Dataset.xlsx")
print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns\n")

# Check what we have
print("Dataset Overview:")
print(df.head())
print("\n")

print("Column Info:")
print(df.info())
print("\n")

print("Missing Values:")
print(df.isna().sum())
print("\n")

# Fix datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.sort_values("timestamp")
print(f"Date Range: {df['timestamp'].min().date()} to {df['timestamp'].max().date()}\n")

# Handle missing values

# For usage_units - interpolate within each region and service group
df["usage_units"] = df.groupby(["region", "service_type"], observed=True)["usage_units"].transform(
    lambda x: x.interpolate().bfill().ffill()
)

# For cost - use average rate if missing
# Calculate avg cost per unit for each service type
compute_rate = df[df['service_type'] == 'Compute']['cost_usd'].sum() / df[df['service_type'] == 'Compute']['usage_units'].sum()
storage_rate = df[df['service_type'] == 'Storage']['cost_usd'].sum() / df[df['service_type'] == 'Storage']['usage_units'].sum()

# Fill missing costs
for idx, row in df[df['cost_usd'].isna()].iterrows():
    if row['service_type'] == 'Compute':
        df.at[idx, 'cost_usd'] = row['usage_units'] * compute_rate
    else:
        df.at[idx, 'cost_usd'] = row['usage_units'] * storage_rate

# For availability - forward fill within groups
df["availability_pct"] = df.groupby(["region", "service_type"], observed=True)["availability_pct"].transform(
    lambda x: x.ffill().bfill()
)

# If any availability still missing, use default
df["availability_pct"] = df["availability_pct"].fillna(99.8)

# Holiday flag - assume 0 if missing
df["is_holiday"] = df["is_holiday"].fillna(0)

print("Missing values after treatment:")
print(df.isna().sum())
print("\n")

# Optimize data types
df["region"] = df["region"].astype("category")
df["service_type"] = df["service_type"].astype("category")
df["is_holiday"] = df["is_holiday"].astype(int)

# Create some useful features

# Utilization percentage
df['utilization_pct'] = (df['usage_units'] / df['provisioned_capacity']) * 100

# Cost per unit
df['cost_per_unit'] = df['cost_usd'] / df['usage_units']

# Buffer capacity
df['buffer_capacity'] = df['provisioned_capacity'] - df['usage_units']

# Time features
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['quarter'] = df['timestamp'].dt.quarter
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

print("New features created:")
print("- utilization_pct")
print("- cost_per_unit")
print("- buffer_capacity")
print("- year, month, quarter")
print("- day_of_week, is_weekend")
print("\n")

# Basic validation checks

# Check capacity constraint
over_capacity = (df['usage_units'] > df['provisioned_capacity']).sum()
print(f"Capacity constraint check:")
print(f"Records where usage > capacity: {over_capacity}")
print("\n")

# Check for duplicates
duplicates = df.duplicated(subset=['timestamp', 'region', 'service_type']).sum()
print(f"Duplicate records: {duplicates}\n")

# Summary statistics
print("Summary Statistics:")
print(f"Total Usage: {df['usage_units'].sum():,.0f} units")
print(f"Total Cost: ${df['cost_usd'].sum():,.2f}")
print(f"Average Utilization: {df['utilization_pct'].mean():.2f}%")
print(f"Average Availability: {df['availability_pct'].mean():.2f}%")
print("\n")

print("By Service Type:")
print(df.groupby('service_type', observed=True)['utilization_pct'].describe())
print("\n")

print("By Region:")
print(df.groupby('region', observed=True)['usage_units'].sum())
print("\n")

# Save cleaned dataset
df.to_csv("azure_demand_cleaned.csv", index=False)
print(f"Cleaned dataset saved: azure_demand_cleaned.csv")
print(f"Final shape: {df.shape[0]} rows, {df.shape[1]} columns")

print("\nMilestone 1 Complete!")


Starting Milestone 1: Data Collection & Preparation

Dataset loaded: 5200 rows, 8 columns

Dataset Overview:
   timestamp        region service_type  usage_units  provisioned_capacity  cost_usd  \
0 2023-04-01   India-South      Storage      10652.0                 14277    214.69   
1 2023-04-01  Europe-North      Compute      12578.0                 15415    701.72   
2 2023-04-01       US-East      Storage          NaN                 10522       NaN   
3 2023-04-01       US-West      Compute       9251.0                 11837    464.30   
4 2023-04-01  Europe-North      Storage       8846.0                 10775    226.57   

   availability_pct  is_holiday  
0             99.74           0  
1             99.82           0  
2             99.90           0  
3             99.97           0  
4             99.85           0  


Column Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 8 columns):
 #   Column                Non-Null C